In [1]:
from langchain import hub
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chat_models import ChatOpenAI
import getpass
import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
# Define a function to format the documents retrieved from the vector store
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [4]:
# Specify the embeddings and directory where the vector store database is located
embedding_function = OpenAIEmbeddings(model="text-embedding-3-large")
persist_directory = "../chroma_db"

# Load the saved vectorstore
vector_store = Chroma(embedding_function=embedding_function, persist_directory=persist_directory)

# Retrieve and generate using the relevant snippets of the vector_store.
retriever = vector_store.as_retriever()

# Use a standard propmt template to perform simple queries on the loaded vectorstore
# TODO: Need to experiment with different prompt templates
prompt = hub.pull("rlm/rag-prompt")
# [HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:")]

# Instantiate the LLM to use
# llm = ChatOpenAI(model="gpt-4o-mini")
llm = ChatOpenAI(model="gpt-4", temperature=1)

# Define the chain
rag_chain = (
    {"context": retriever | format_docs, 
     "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print(prompt)

d:\Projects\group3-final-project\.venv\Lib\site-packages\langsmith\client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [11]:
# Start asking questions

#question = """The lack of police presence and code enforcement is sending a growing message that these violations are not important. Second item: affordable denver and wanting more information about how the tax will accomplish the goals set by Mayor."""
question = """I want information on getting a compost bim. I have submitted case number: 9578014."""

rag_chain.invoke(question)

"The information available indicates that getting a compost bin is managed on a district-by-district basis by Solid Waste. Unfortunately, it's not possible for you to pick up a compost bin yourself, and your district (SWC District 9) is scheduled to receive compost bins in 2025. This schedule is apparently strict, with special emphasis given to helping customers understand the use of the compost bins when they are delivered."